<a href="https://colab.research.google.com/github/dev-alexandremarcet/mvp_ml_students/blob/main/MVP_ML_Bolsas_de_Estudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MVP de Machine Learning - Alexandre Peixoto Marcet**

  Este MVP é um modelo de machine learning de classificação. Ele faz a predição de um determinado aluno (ingressante numa faculdade particular) receber bolsa de estudos ou não.

# 1 - Definição do Problema

O problema se resume em fazer a previsão se um aluno vai receber uma bolsa de estudos, considerando que esse aluno está ingressando numa determinada universidade privada. Essa previsão deverá ser feita a partir de algumas informações do aluno, tais como: renda per capita familiar, notas no vestibular, participação em programas sociais do governo e premiações em olimpíadas do conhecimento. Essas informações servem de base para o estabelecimento de critérios para a concessão de bolsas pela referida universidade privada.
Os dados estão restritos a informações que não identificam os alunos. Essa foi a condição para a obtenção, uso e divulgação dos dados neste MVP.
O dataset possui os seguintes atributos:

[0] - num_inscricao-->número de inscrição do aluno no processo seletivo para concessão de bolsas de estudo

[1] - nota_mat-->nota do aluno na área de Matemática no vestibular

[2] - nota_linguagens-->nota do aluno na área de Linguagens no vestibular

[3] - nota_ciencias_hum-->nota do aluno na área de Ciências Humanas no vestibular

[4] - nota_ciencias_nat-->nota do aluno na área de Ciências da Natureza no vestibular

[5] - nota_redacao-->nota do aluno em redação no vestibular

[6] - renda_per_capita_fam-->renda per capita do núcleo familiar do aluno

[7] - premio_olimp_cohecimento-->atributo para a verificação se o aluno recebeu premiação em qualquer olimpíada do conhecimento(1 se recebeu, 0 se não recebeu)

[8] - prog_social_gov-->atributo para a verificação se o aluno (ou a família dele) participa de algum programa social de transferência de renda do governo(municipal,estadual ou federal) (1 se o aluno ou a família participa, 0 se não participa)


# 2 - Preparação dos Dados

## 2.1 - Importação de modelos, classificadores, bibliotecas e algoritmos

In [13]:
# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split # para particionar em bases de treino e teste (holdout)
from sklearn.model_selection import KFold # para preparar os folds da validação cruzada
from sklearn.model_selection import StratifiedKFold # para preparar os folds com estratificação
from sklearn.model_selection import cross_val_score # para executar a validação cruzada
from sklearn.metrics import accuracy_score # para a exibição da acurácia do modelo
from sklearn.pipeline import Pipeline # para preparar os pipelines
from sklearn.linear_model import LogisticRegression # algoritmo de Regressão Logística
from sklearn.neighbors import KNeighborsClassifier # algoritmo KNN
from sklearn.tree import DecisionTreeClassifier # algoritmo Árvore de Classificação
from sklearn.naive_bayes import GaussianNB # algoritmo Naive Bayes
from sklearn.svm import SVC # algoritmo SVM
from sklearn.ensemble import ExtraTreesClassifier # ExtraTrees para a feature selection Importância de Atributos e ensemble Bagging
from sklearn.ensemble import BaggingClassifier # para ensemble Bagging
from sklearn.ensemble import RandomForestClassifier # para ensemble Bagging
from sklearn.ensemble import VotingClassifier # para ensemble Voting
from sklearn.ensemble import AdaBoostClassifier # para ensemble Boosting
from sklearn.ensemble import GradientBoostingClassifier # para ensemble Boosting

## 2.2 - Carregamento do Dataset

In [ ]:
# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/dev-alexandremarcet/mvp_ml_students/main/students-dataset.csv"

# Lê o arquivo
dataset = pd.read_csv(url, delimiter=',')

# Mostra as primeiras linhas do dataset
dataset.head()

## 2.3 - Separação do dataset em conjunto de treino e conjunto de teste (usando holdout com estratificação) e fazendo a validação cruzada

In [15]:
# Preparação dos dados
test_size = 0.30 # definindo o tamanho do conjunto de teste
seed = 7 # definindo a semente aleatória

# Separação em conjuntos de treino e teste (holdout)
array = dataset.values
X = array[:,0:9] # atributos
Y = array[:,9] # classe (recebe_bolsa)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, shuffle=True, random_state=seed, stratify=Y) # efetua a divisão dos dados em conjuntos de treino e teste (holdout)

# Definindo parâmetros e criando os folds para a validação cruzada
scoring = 'accuracy'
num_particoes = 10 # número de folds da validação cruzada
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # faz o particionamento em 10 folds


## 2.4 - Verificando a importância de atributos

In [ ]:
# Criação do modelo para a seleção de atributos
modelo = ExtraTreesClassifier(n_estimators=100)
modelo.fit(X,Y)

# Exibe os atributos originais
print("\Atributos Originais:", dataset.columns[0:9])

# Exibe a pontuação de importância para cada atributo
print(modelo.feature_importances_)

### Observação: Como apenas dois atributos possuem valores maiores do que 0,11 (aproximadamente a média das pontuações), nenhum atributo foi descartado. Além disso, o número de atributos do dataset é bem pequeno.

# 3 - Modelagem e Treinamento

## 3.1 - Criação de modelos básicos e *ensembles*


In [17]:
# Modelagem

# Definindo uma seed global para esta célula de código
np.random.seed(seed)

# Criando uma lista para armazenar os modelos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('LR', LogisticRegression(max_iter=200)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Definindo os parâmetros do classificador base para o BaggingClassifier
base = DecisionTreeClassifier()
num_trees = 100
max_features = 3

# Criando os modelos para o VotingClassifier
bases = []
model1 = LogisticRegression(max_iter=200)
bases.append(('logistic', model1))
model2 = DecisionTreeClassifier()
bases.append(('cart', model2))
model3 = SVC()
bases.append(('svm', model3))

# Criando os ensembles e adicionando-os na lista de modelos
models.append(('Bagging', BaggingClassifier(base_estimator=base, n_estimators=num_trees)))
models.append(('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)))
models.append(('Ada', AdaBoostClassifier(n_estimators=num_trees)))
models.append(('GB', GradientBoostingClassifier(n_estimators=num_trees)))
models.append(('Voting', VotingClassifier(bases)))

### Observação: Foram criados modelos básicos e *ensembles* para disponibilizar mais modelos para treinamento, com o objetivo de resolver o problema usando o melhor modelo possível.

## 3.2 - Avaliação dos modelos básicos e *ensembles*

In [ ]:
# Listas para armazenar os resultados
results = []
names = []

# Avaliando um modelo por vez
for name, model in models:
  cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()) # média e desvio padrão dos 10 resultados da validação cruzada
  print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(15,10))
fig.suptitle('Comparação da Acurácia dos Modelos')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

## 3.3 - Criação de modelos com dados padronizados e com dados normalizados

In [ ]:
np.random.seed(7) # definindo uma semente global para este bloco

# Listas para armazenar os armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []


# Criando os elementos do pipeline

# Algoritmos que serão utilizados
reg_log = ('LR', LogisticRegression(max_iter=200))
knn = ('KNN', KNeighborsClassifier())
cart = ('CART', DecisionTreeClassifier())
naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())
bagging = ('Bag', BaggingClassifier(base_estimator=base, n_estimators=num_trees))
random_forest = ('RF', RandomForestClassifier(n_estimators=num_trees, max_features=max_features))
extra_trees = ('ET', ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features))
adaboost = ('Ada', AdaBoostClassifier(n_estimators=num_trees))
gradient_boosting = ('GB', GradientBoostingClassifier(n_estimators=num_trees))
voting = ('Voting', VotingClassifier(bases))

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())


# Montando os pipelines

# Dataset original
pipelines.append(('LR-orig', Pipeline([reg_log])))
pipelines.append(('KNN-orig', Pipeline([knn])))
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))
pipelines.append(('Bag-orig', Pipeline([bagging])))
pipelines.append(('RF-orig', Pipeline([random_forest])))
pipelines.append(('ET-orig', Pipeline([extra_trees])))
pipelines.append(('Ada-orig', Pipeline([adaboost])))
pipelines.append(('GB-orig', Pipeline([gradient_boosting])))
pipelines.append(('Vot-orig', Pipeline([voting])))

# Dataset Padronizado
pipelines.append(('LR-padr', Pipeline([standard_scaler, reg_log])))
pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))
pipelines.append(('Bag-padr', Pipeline([standard_scaler, bagging])))
pipelines.append(('RF-padr', Pipeline([standard_scaler, random_forest])))
pipelines.append(('ET-padr', Pipeline([standard_scaler, extra_trees])))
pipelines.append(('Ada-padr', Pipeline([standard_scaler, adaboost])))
pipelines.append(('GB-padr', Pipeline([standard_scaler, gradient_boosting])))
pipelines.append(('Vot-padr', Pipeline([standard_scaler, voting])))

# Dataset Normalizado
pipelines.append(('LR-norm', Pipeline([min_max_scaler, reg_log])))
pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))
pipelines.append(('Bag-norm', Pipeline([min_max_scaler, bagging])))
pipelines.append(('RF-norm', Pipeline([min_max_scaler, random_forest])))
pipelines.append(('ET-norm', Pipeline([min_max_scaler, extra_trees])))
pipelines.append(('Ada-norm', Pipeline([min_max_scaler, adaboost])))
pipelines.append(('GB-norm', Pipeline([min_max_scaler, gradient_boosting])))
pipelines.append(('Vot-norm', Pipeline([min_max_scaler, voting])))

## 3.4 - Avaliação dos modelos com dados padronizados e com dados normalizados

In [ ]:
# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
    print(msg)

# Boxplot de comparação dos modelos
fig = plt.figure(figsize=(25,6))
fig.suptitle('Comparação dos Modelos - Dataset orginal, padronizado e normalizado')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names, rotation=90)
plt.show()

### Observação: Não foi feita a otimização de hiperparâmetros porque a acurácia do modelo (GradientBoostingClassifier-Original) é aproximadamente de 97%. O que justifica essa decisão.

# 4 - Avaliação de Resultados

## 4.1 - Avaliação do melhor modelo (GradientBoostingClassifier) no Conjunto de Testes



In [ ]:
# Avaliação do modelo com o conjunto de testes

# Preparação do modelo
scaler = StandardScaler().fit(X_train) # ajuste do scaler com o conjunto de treino
rescaledX = scaler.transform(X_train) # aplicação da padronização no conjunto de treino
model = GradientBoostingClassifier(n_estimators=num_trees)
model.fit(rescaledX, Y_train)

# Estimativa da acurácia no conjunto de teste
rescaledTestX = scaler.transform(X_test) # aplicação da padronização no conjunto de teste
predictions = model.predict(rescaledTestX)
print(accuracy_score(Y_test, predictions))

## 4.2 - Avaliação do melhor modelo

In [ ]:
# Preparação do modelo com TODO o dataset
scaler = StandardScaler().fit(X) # ajuste do scaler com TODO o dataset
rescaledX = scaler.transform(X) # aplicação da padronização com TODO o dataset
model.fit(rescaledX, Y)

## 4.3 - Novos dados para teste com o modelo GradientBoostingClassifier

In [ ]:
# Novos dados - não sabemos a classe!
data = {'num_inscricao':  [901, 902, 903, 904, 905, 906, 907, 908, 909, 910],
        'nota_mat': [90, 65, 85, 77, 86, 91, 75, 98, 93, 81],
        'nota_linguagens': [95, 79, 83, 71, 59, 88, 86, 77, 85, 82],
        'nota_ciencias_hum': [92, 81, 86, 92, 61, 97, 92, 99, 90, 83],
        'nota_ciencias_nat': [100, 64, 84, 98, 72, 99, 78, 72, 77, 84],
        'nota_redacao': [94, 90, 95, 99, 53, 86, 79, 95, 78, 79],
        'renda_fam_per_capita': [1500, 1300, 1200, 1180, 1700, 1800, 980, 1020, 1650, 1010],
        'premio_olimp_conhecimento': [1, 0, 1, 0, 0, 1, 0, 1, 1, 0],
        'prog_social_gov': [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
        }

atributos = ['num_inscricao', 'nota_mat', 'nota_linguagens', 'nota_ciencias_hum', 'nota_ciencias_nat', 'nota_redacao', 'renda_fam_per_capita', 'premio_olimp_conhecimento', 'prog_social_gov']
entrada = pd.DataFrame(data, columns=atributos)

array_entrada = entrada.values
X_entrada = array_entrada[:,0:9].astype(float)

# Padronização nos dados de entrada usando o scaler utilizado em X
rescaledEntradaX = scaler.transform(X_entrada)
print(rescaledEntradaX)

## 4.4 - Predição com o modelo GradientBoostingClassifier

In [ ]:
# Predição de classes dos dados de entrada
saidas = model.predict(rescaledEntradaX)
print(saidas)

### Considerações Finais: Comparando os resultados de diferentes modelos, podemos observar que o modelo GradientBoostingClassifier(Original) foi o melhor com acurácia de aproximadamente 97%. Esse modelo obteve praticamente os mesmos (e melhores) valores com a padronização e com a normalização. Não foi verificado overfiting nem underfiting. Após o treinamento do modelo GradientBoostingClassifier, ele foi utilizado com toda a base de treino, com a base de teste e com todo o dataset.